<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_13_8_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

8- analogous to task 3, but implementing a Bayesian hierarchical regression (with
continuous outcome variable) with sex as binary higher level to predict subject age
(zscored).

In [ ]:
%%capture
!pip install nilearn
!pip install pymc

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
from sklearn.preprocessing import StandardScaler
import nibabel as nib

NOTE: for this exercise I used PyMC version 5 instead of PyMC3 because the model ran better on the former package. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc as pm
import scipy.stats as stats
import arviz as az
from pymc import Bernoulli, Model, HalfCauchy
from pymc import invlogit, sample, summary
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.cross_decomposition import CCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR

In [ ]:
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()
atlas_nii = index_img(crad['scorr_mean'], 10) # we will set this to 10 for these exercises in order to have 106 rois, which is close to the 100 rois in Exercises 1-5
masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = StandardScaler().fit_transform(brain_data.ext_vars.age[:, None])[:, 0] # gives subject age on standard units after z-scoring

In [ ]:
# We delete the last 6 columns from "input_variables" so that we have the same number of input_features as subjects (100, 100)
input_variables = input_variables[:, :-6]

In [ ]:
# We extract the sex information from the dataset, binarize it, and count the number of females
male_female = brain_data.ext_vars.mf
male_female = np.where(np.isin(male_female, 'M'), 0, 1)
np.count_nonzero(male_female == 1)

64

Since there are more females than males, we will have to factor that in when we create the hyperprior category based on sex. 

In [ ]:
# This time around we have concrete information about the proportions of the categorical hyperprior. Let's implement it in a model
with pm.Model() as model:
  
    # We define a HalfCauchy hyperprior for variance component of lower-level region slopes
    hc = pm.HalfCauchy('hc', beta=1, shape=100)

    # We define sex as a categorical distribution with 2 categories (male, female) with unequal probabilities this time
    sex = pm.Categorical('sex', p=np.array([0.36, 0.64]), observed=male_female)
   
    # We define the intercept term as a normal distribution with priors mu=0 and sigma=1
    intercept = pm.Normal('intercept', mu=0, sigma=1)
    
    # We define the bottom-level region slopes (or betas) as a normal distribution with priors (mu=0, sigma=hyperprior)
    beta_coefficients = pm.Normal('beta_coefficients', mu=0, sigma=hc, shape=100)
    
    # We establish the desired linear model using input_variables as our "X"es and the required model parameters
    linear_model = intercept + input_variables @ beta_coefficients + sex
    
    # We define the likelihood as another Normal distribution with mean equal to probabilities and sigma=1
    likelihood = pm.Normal('likelihood', mu=linear_model, sigma=1, observed=output_variable)

    # We sample from the posterior distribution to estimate the unseen variables using 500 MCMC draws.
    trace = pm.sample(500, return_inferencedata=True)
    # We call on the arviz package to help us define the summary table
    stats = az.summary(trace, hdi_prob=0.9)


ERROR:pymc:There were 134 divergences after tuning. Increase `target_accept` or reparameterize.
ERROR:pymc:There were 303 divergences after tuning. Increase `target_accept` or reparameterize.


In [ ]:
# Let's take a look at the summary table
from google.colab import data_table
data_table.DataTable(stats, include_index=True, num_rows_per_page=201)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-0.635,0.095,-0.777,-0.467,0.004,0.003,732.0,578.0,1.00
beta_coefficients[0],0.102,0.291,-0.336,0.598,0.016,0.012,356.0,385.0,1.02
beta_coefficients[1],-0.127,0.320,-0.687,0.348,0.020,0.014,259.0,308.0,1.00
beta_coefficients[2],0.041,0.275,-0.355,0.582,0.013,0.010,461.0,398.0,1.00
beta_coefficients[3],-0.056,0.266,-0.521,0.359,0.013,0.009,401.0,448.0,1.00
...,...,...,...,...,...,...,...,...,...
hc[95],0.917,3.215,0.003,1.556,0.158,0.112,353.0,361.0,1.01
hc[96],0.778,1.409,0.003,1.504,0.056,0.040,442.0,304.0,1.00
hc[97],0.758,0.981,0.002,1.697,0.041,0.030,435.0,327.0,1.01
hc[98],0.798,1.474,0.006,1.682,0.066,0.047,400.0,431.0,1.01
